In [3]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

amap=pd.read_hdf('Processed_Data/amap.h5',key='1210update')
NEWday=pd.read_excel('./DataSource/DailyUpdateForTG.xlsx')
NEWday=pd.melt(NEWday,id_vars=['城市'],var_name='date', value_name='traffic')
updated=pd.concat([amap,NEWday]).drop_duplicates()
updated.to_hdf('Processed_Data/amap.h5',key='1210update')

amap=updated
amap['date']=pd.to_datetime(amap['date'])
amap['WeekDay']=amap['date'].dt.strftime('%A')


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d.date())[-5:]
def getMon(d):
    return str(d.date()).split('-')[1]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)
amap['mon']=amap['date'].apply(getMon)

amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

amap_2019=amap[amap['yr'].str.contains('2019')]
# amap_2019['traffic_last30d_mean'] = amap_2019.groupby('城市')['traffic'].transform(lambda x: x.rolling(30, 1).mean()) # 当的traffic 变化
amap_2019=amap_2019.groupby(by=['城市','mon','WeekDay'])['traffic'].mean().reset_index() # 当的traffic 变化
amap_2019=amap_2019.rename(columns={"traffic":"traffic_19_same_wd_mean"})

amap_2022=amap[amap['yr'].str.contains('2022')]
amap_2022=amap_2022.merge(amap_2019,how='left',on=['城市','mon','WeekDay'])

amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2022['traffic_change']=amap_2022['traffic_last7d_mean']/amap_2022.groupby('城市')['traffic_last7d_mean'].transform(lambda x: x.shift(periods=7))-1

amap_2022['traffic_status']=amap_2022['traffic']/amap_2022['traffic_19_same_wd_mean']

amap_2022.tail(2)

,城市,date,traffic,WeekDay,yr,dt,mon,traffic_19_same_wd_mean,traffic_last7d_mean,traffic_change,traffic_status
35998,潍坊,2022-12-28,1.37,Wednesday,2022,12-28,12,1.60,1.351429,-0.014583,0.856250
35999,肇庆,2022-12-28,1.28,Wednesday,2022,12-28,12,1.46,1.274286,-0.094416,0.876712


In [4]:
# 2.0 TG 流动性影响 by 人口拆分
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])


df_amap=amap_2022.merge(city_popu,on='城市',how='inner')

df_amap['traffic_status_lastXd'] = df_amap.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())

def catgorize2(g):
    g=g-1
    level='5 社会面流动性无影响'
    g=float(g)
    if g <=-0.25:
        level= '0 社会面基本停滞'
    elif -0.25<g<=-0.15:
        level ='1 社会面流动性显著降低'
    elif -0.15<g<=-0.05:
        level ='4 社会面流动性轻微降低'
    elif g>-0.05:
        level ='5 社会面流动性无影响'
    return level

df_amap['traffic_change_level']=df_amap['traffic_status_lastXd'].apply(catgorize2)

df_amap['date']=pd.to_datetime(df_amap['date'])

pv_value=pd.pivot_table(df_amap, values='常住人口', index=['date'],columns=["traffic_change_level"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 社会面基本停滞', '1 社会面流动性显著降低', '4 社会面流动性轻微降低', '5 社会面流动性无影响']]
pv_value=pv_value.fillna(0)
pv_value.tail(22).to_clipboard(header=None)
pv_value.tail(3)

traffic_change_level,sum,0 社会面基本停滞,1 社会面流动性显著降低,4 社会面流动性轻微降低,5 社会面流动性无影响
date,,,,,
2022-12-26,1,0.085121,0.606035,0.278422,0.030422
2022-12-27,1,0.096341,0.579016,0.236965,0.087678
2022-12-28,1,0.092443,0.495174,0.324705,0.087678


In [24]:
# df_covid=pd.read_hdf(r'BaiduFever.h5',key='1220')
# df_covid['BDfever_last7Avg']=df_covid.groupby('城市')['BDfever'].transform(lambda x: x.rolling(7, 7).mean())
# df_covid.tail(2)

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid.tail(2)

,city,progress,peak,end,城市,常住人口,progress_2
295,聊城,回落至34%,2022/12/20,2023/03/07,聊城,201,34
296,宿迁,回落至41%,2022/12/21,2023/03/08,宿迁,310,41


In [25]:
# 2.1 TY Amap traffic change WoW vs.  covid rank
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

dfo=amap_2022.merge(df_covid,on=['城市'],how='left').merge(city_popu,on='城市',how='inner')
dfo['date']=dfo['date'].astype('string') 
dfo['covid_rank']=dfo.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=dfo[dfo['date']==riqi][['城市','covid_rank','traffic_change']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)

print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-12-26


In [26]:
# 2.2 TY Amap traffic change /2019 vs. TY covid rank traffic_status_lastXd
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

amap_2022['traffic_status_lastXd'] = amap_2022.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())
dfo=amap_2022.merge(df_covid,on=['城市'],how='left').merge(city_popu,on='城市',how='inner')

dfo['date']=dfo['date'].astype('string') 
dfo['covid_rank']=dfo.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=dfo[dfo['date']==riqi][['城市','covid_rank','traffic_status_lastXd']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)

print(riqi)

2022-12-26
